If you are opening this notebook in Colab, you will need to upload `requirements.txt` to your Colab file system

In [ ]:
! pip install -r requirements.txt

  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-_mmcjmem
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-_mmcjmem
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done


In [ ]:
import transformers

print(transformers.__version__)

4.20.0


# Fine-tuning a model on a translation task

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a sequence-to-sequence version in the Transformers library. Here we picked the [`Helsinki-NLP/opus-mt-tc-big-en-tr`](https://huggingface.co/Helsinki-NLP/opus-mt-tc-big-en-tr) checkpoint.

In [ ]:
model_checkpoint = "Helsinki-NLP/opus-mt-tc-big-en-tr"

## Loading the dataset

We will use the [Datasets](https://github.com/huggingface/datasets) library to download the data. This can be easily done with the functions `load_dataset`. We use the English/Turkish part of the WMT dataset here.

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("wmt18", "tr-en")

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 205756
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 3007
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3000
    })
})

To access an actual element, you need to select a split first, then give an index:

In [ ]:
raw_datasets["train"][0]

{'translation': {'en': "Kosovo's privatisation process is under scrutiny",
  'tr': "Kosova'nın özelleştirme süreci büyüteç altında"}}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["train"])

,translation
0,"{'en': 'The observatory is the most modern of its kind in southeastern Europe and has state-of-the-art equipment imported from the United States.', 'tr': 'Güneydoğu Avrupa'da türünün en çağdaş örneği olan gözlemevi, ABD'den ithal edilmiş en son teknolojiye sahip cihazlarla donatıldı.'}"
1,"{'en': 'However, the most important thing is not to give up art at any moment.', 'tr': 'Fakat yine de en önemlisi, sanattan hiçbir zaman vazgeçmemek.'}"
2,"{'en': 'The album will go on sale in all of the former Yugoslav republics.', 'tr': 'Albüm bütün eski Yugoslav cumhuriyetlerinde satışa sunulacak.'}"
3,"{'en': 'I cannot explain that to anyone,"" de Hoop Scheffer said during talks with Serbian, Montenegrin and Serbian-Montenegrin officials.', 'tr': 'Bunu kimseye açıklayamıyorum,"" dedi.'}"
4,"{'en': 'In the IMF's view, maintaining economic growth and low inflation are the country's two largest macroeconomic challenges.', 'tr': 'IMF'ye göre, ekonomik büyüme ve düşük enflasyonun korunması ülkenin önünde bulunan iki büyük makroekonomik engel.'}"


 It is also necessary to get the metrics to evaluate our model performance. The HuggingFace Evaluate library's `evaluate.combine()` method allows us to use several metrics at the same time.

In [ ]:
import evaluate

mt_metrics = evaluate.combine(
    ["bleu", "google_bleu", "meteor", "bleurt"], force_prefix=True
)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


You can call its `compute` method with your predictions and labels, which need to be list of decoded strings:

In [ ]:
fake_preds = ["hello here I am your father", "general kenobi"]
fake_labels = ["hello there I am your father", "general yoda"]
mt_metrics.compute(predictions=fake_preds,
                             references=fake_labels)

{'bleu_bleu': 0.49999999999999994,
 'bleu_precisions': [0.75, 0.5, 0.5, 0.3333333333333333],
 'bleu_brevity_penalty': 1.0,
 'bleu_length_ratio': 1.0,
 'bleu_translation_length': 8,
 'bleu_reference_length': 8,
 'google_bleu_google_bleu': 0.5714285714285714,
 'meteor_meteor': 0.5283333333333333,
 'bleurt_scores': [0.8902602195739746, -0.13236568868160248]}

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

loading file https://huggingface.co/Helsinki-NLP/opus-mt-tc-big-en-tr/resolve/main/source.spm from cache at /root/.cache/huggingface/transformers/b7a0846987b69fdf8eecc3508a81e65aeb6043b3b008c6ee85503c428a3dfa4e.7351a75aa59476a1a3765633e58237b8e1b0987a16301f1509fc48b4bd26311a
loading file https://huggingface.co/Helsinki-NLP/opus-mt-tc-big-en-tr/resolve/main/target.spm from cache at /root/.cache/huggingface/transformers/e88148a40a553b5057a492d8ffef741c41825105beadc466968c5ad3c858a8ab.51e12b89fb2e29edf6aed526c716ed6898db953411273a25978d3d2b1697089d
loading file https://huggingface.co/Helsinki-NLP/opus-mt-tc-big-en-tr/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/e7cf6363a117c2c0f288c9887f897dd0d5f767e353f12d9b6940982374f36e2d.54a9d2ab089d3c0f9bb905f36d8cc814dcd611395cc4018103cd015b6b19dd36
loading file https://huggingface.co/Helsinki-NLP/opus-mt-tc-big-en-tr/resolve/main/target_vocab.json from cache at None
loading file https://huggingface.co/Helsinki-NLP/opu

For the mBART tokenizer (like we have here), we need to set the source and target languages (so the texts are preprocessed properly).

In [ ]:
if "mbart" in model_checkpoint:
    tokenizer.src_lang = "en-XX"
    tokenizer.tgt_lang = "tr-TR"

You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
tokenizer("Hello, this one sentence!")

{'input_ids': [24170, 14, 50518, 37720, 45280, 21, 43741], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[24170, 14, 50518, 37720, 45280, 21, 43741], [50520, 27073, 4426, 45280, 33, 43741]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

To prepare the targets for our model, we need to tokenize them inside the `as_target_tokenizer` context manager. This will make sure the tokenizer uses the special tokens corresponding to the targets:

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[24170, 14, 50518, 36614, 35836, 45198, 50069, 10272, 21, 43741], [50520, 27073, 90, 4411, 50426, 45198, 50069, 10272, 33, 43741]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


If you are using one of the five T5 checkpoints that require a special prefix to put before the inputs, you should adapt the following cell.

In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Turkish: "
else:
    prefix = ""

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [ ]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "tr"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[30475, 46, 43740, 90, 40936, 27109, 40989, 27073, 52359, 44770, 43741], [30475, 27073, 49203, 2136, 23602, 32605, 5431, 27628, 90, 40936, 27109, 40989, 26103, 32153, 36851, 42280, 11977, 33, 43741]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[30474, 46, 36205, 38702, 48710, 9525, 16546, 9560, 3836, 43741], [30474, 14, 49928, 16802, 46100, 27161, 38702, 48715, 26190, 3663, 33, 43741]]}

To apply this function on all the pairs of sentences in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

loading configuration file https://huggingface.co/Helsinki-NLP/opus-mt-tc-big-en-tr/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/b5bc1a68975e95953693dc3b566539668d0255026435b892c41a89d41e6fc8db.df24946fd9cff298d60ef3ca482c189245ad9f081df860ddb1a5e152c54ae469
Model config MarianConfig {
  "_name_or_path": "Helsinki-NLP/opus-mt-tc-big-en-tr",
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      57059
    ]
  ],
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 57059,
  "decoder_vocab_size": 57060,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 43741,
  "forced_eos_token_id": 43741,
  "init

Note that  we don't get a warning like in our classification example. This means we used all the weights of the pretrained model and there is no randomly initialized head in this case.

To instantiate a `Seq2SeqTrainer`, we will need to define three more things. The most important is the [`Seq2SeqTrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    eval_accumulation_steps=16,
    gradient_accumulation_steps=16,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    # adafactor=True,
    # optim = "adafactor",
    # warmup_steps=50,
    # lr_scheduler_type = "linear",
    # report_to="tensorboard",
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the cell and customize the weight decay. Since the `Seq2SeqTrainer` will save the model regularly and our dataset is quite large, we tell it to make three saves maximum. Lastly, we use the `predict_with_generate` option (to properly generate summaries) and activate mixed precision training (to go a bit faster).

Then, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels:

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [ ]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = mt_metrics.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["bleu_bleu"], "google_bleu": result["google_bleu_google_bleu"],
              "meteor": result["meteor_meteor"], "bleurt": np.mean(result["bleurt_scores"])}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using cuda_amp half precision backend


We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation. If translation are not expected by `MarianMTModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 205756
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 16
  Total optimization steps = 803


Epoch,Training Loss,Validation Loss,Bleu,Google Bleu,Meteor,Bleurt,Gen Len
0,1.074800,1.455875,0.245200,0.289000,0.515800,0.104300,23.888600


Saving model checkpoint to opus-mt-tc-big-en-tr-finetuned-en-to-tr/checkpoint-500
Configuration saved in opus-mt-tc-big-en-tr-finetuned-en-to-tr/checkpoint-500/config.json
Model weights saved in opus-mt-tc-big-en-tr-finetuned-en-to-tr/checkpoint-500/pytorch_model.bin
tokenizer config file saved in opus-mt-tc-big-en-tr-finetuned-en-to-tr/checkpoint-500/tokenizer_config.json
Special tokens file saved in opus-mt-tc-big-en-tr-finetuned-en-to-tr/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation. If translation are not expected by `MarianMTModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3007
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=803, training_loss=1.0598700536440497, metrics={'train_runtime': 1041.0367, 'train_samples_per_second': 197.645, 'train_steps_per_second': 0.771, 'total_flos': 1.2626132799062016e+16, 'train_loss': 1.0598700536440497, 'epoch': 1.0})

You can call tensorboard in order to visualize the results of the training process, just uncomment the following cell and run it.

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir /content/opus-mt-tc-big-en-tr-finetuned-en-to-tr/runs

#Model Testing

Finally, you can now test your model performance by executing the following cells and try to translate what you want.

In [ ]:
trainer.save_model("tf_model/")

Saving model checkpoint to tf_model/
Configuration saved in tf_model/config.json
Model weights saved in tf_model/pytorch_model.bin
tokenizer config file saved in tf_model/tokenizer_config.json
Special tokens file saved in tf_model/special_tokens_map.json


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained("tf_model/")

loading file https://huggingface.co/Helsinki-NLP/opus-mt-tc-big-en-tr/resolve/main/source.spm from cache at /root/.cache/huggingface/transformers/b7a0846987b69fdf8eecc3508a81e65aeb6043b3b008c6ee85503c428a3dfa4e.7351a75aa59476a1a3765633e58237b8e1b0987a16301f1509fc48b4bd26311a
loading file https://huggingface.co/Helsinki-NLP/opus-mt-tc-big-en-tr/resolve/main/target.spm from cache at /root/.cache/huggingface/transformers/e88148a40a553b5057a492d8ffef741c41825105beadc466968c5ad3c858a8ab.51e12b89fb2e29edf6aed526c716ed6898db953411273a25978d3d2b1697089d
loading file https://huggingface.co/Helsinki-NLP/opus-mt-tc-big-en-tr/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/e7cf6363a117c2c0f288c9887f897dd0d5f767e353f12d9b6940982374f36e2d.54a9d2ab089d3c0f9bb905f36d8cc814dcd611395cc4018103cd015b6b19dd36
loading file https://huggingface.co/Helsinki-NLP/opus-mt-tc-big-en-tr/resolve/main/target_vocab.json from cache at None
loading file https://huggingface.co/Helsinki-NLP/opu

In [ ]:
input_text = "She told me that she had seen a very interesting movie"

tokenized = tokenizer([input_text], return_tensors='pt')
out = model.generate(**tokenized, max_length=128)
print(out)

tensor([[57059,  6444, 11667, 25745,  7920, 19775, 27749, 36119, 47283,    33,
         43741]])


In [ ]:
with tokenizer.as_target_tokenizer():
  print(tokenizer.decode(out[0], skip_special_tokens=True))

Bana çok ilginç bir film izlediğini söyledi.
